# Flood Area Segmentation — Full Training Pipeline

This notebook contains an end-to-end pipeline for segmentation of flood-affected areas.
It includes data loading, preprocessing, a DeepLabV3+ model, losses/metrics (Dice, IoU),
tf.data pipeline, augmentations, focal loss option, Keras Tuner example, callbacks, Grad-CAM style explainability,
and evaluation/visualization helpers.

Run in Kaggle/Colab; adjust `root_path` to point to your dataset.


## How to Run This Notebook
1. Ensure you have TensorFlow, Keras, albumentations, and other required libraries installed.
2. Place your dataset in the expected directory structure.
3. Run the notebook sequentially from top to bottom.
4. GPU is recommended for training DeepLabV3.


## Expected Runtime
- Data loading & preprocessing: ~1–3 minutes depending on dataset size.
- Model training: ~10–30 minutes per epoch on GPU; much longer on CPU.
- Evaluation & visualization: ~1 minute.


In [6]:
# 1) Imports & reproducibility
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print('TensorFlow', tf.__version__)


TensorFlow 2.20.0


In [7]:
# 2) Paths & config - set root_path to your dataset location
root_path = 'D:\D\images'
IMAGE_FOLDER = os.path.join(root_path, 'Image')
MASK_FOLDER  = os.path.join(root_path, 'Mask')
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 60
MODEL_NAME = 'DeepLabV3plus_Flood'


In [8]:
# 3) Robust loading utilities
from tensorflow.keras.utils import load_img, img_to_array

def load_image(path: str, size=IMAGE_SIZE, mask: bool = False):
    if mask:
        pil = load_img(path, color_mode='grayscale', target_size=size)
        arr = img_to_array(pil).astype(np.float32) / 255.0
        return arr
    else:
        pil = load_img(path, color_mode='rgb', target_size=size)
        arr = img_to_array(pil).astype(np.float32) / 255.0
        return arr

def collect_paths(image_folder=IMAGE_FOLDER, mask_folder=MASK_FOLDER):
    image_paths = sorted(glob(os.path.join(image_folder, '*.jpg')))
    mask_paths  = [os.path.join(mask_folder, os.path.basename(p).rsplit('.',1)[0] + '.png') for p in image_paths]
    filtered_image_paths = []
    filtered_mask_paths = []
    for ip, mp in zip(image_paths, mask_paths):
        if os.path.exists(mp):
            filtered_image_paths.append(ip)
            filtered_mask_paths.append(mp)
        else:
            print('Missing mask for:', ip)
    return filtered_image_paths, filtered_mask_paths

image_paths, mask_paths = collect_paths()
print('Found', len(image_paths), 'pairs')


Found 290 pairs


In [18]:
# 4) tf.data pipeline with augmentation
AUTOTUNE = tf.data.AUTOTUNE

def _read_numpy(image_path, mask_path):
    image_path = image_path.numpy().decode("utf-8")
    mask_path = mask_path.numpy().decode("utf-8")

    img = load_image(image_path, size=IMAGE_SIZE, mask=False)
    msk = load_image(mask_path, size=IMAGE_SIZE, mask=True)
    return img, msk

def make_tf_dataset(image_paths, mask_paths, batch_size=BATCH_SIZE, is_training=True):
    ds = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    if is_training:
        ds = ds.shuffle(1024, seed=SEED)

    def _py_load(img_p, msk_p):
        img, msk = tf.py_function(_read_numpy, inp=[img_p, msk_p], Tout=[tf.float32, tf.float32])
        img.set_shape([IMAGE_SIZE[0], IMAGE_SIZE[1], 3])
        msk.set_shape([IMAGE_SIZE[0], IMAGE_SIZE[1], 1])
        return img, msk

    ds = ds.map(_py_load, num_parallel_calls=AUTOTUNE)

    def _augment(image, mask):
        if is_training:
            if tf.random.uniform(()) > 0.5:
                image = tf.image.flip_left_right(image)
                mask  = tf.image.flip_left_right(mask)
            if tf.random.uniform(()) > 0.5:
                image = tf.image.flip_up_down(image)
                mask  = tf.image.flip_up_down(mask)
            image = tf.image.random_brightness(image, 0.06)
            image = tf.image.random_contrast(image, 0.95, 1.05)
        return image, mask

    ds = ds.map(lambda x,y: _augment(x,y), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

n = len(image_paths)
split_at = int(n * 0.8)
train_img_paths = image_paths[:split_at]
train_mask_paths = mask_paths[:split_at]
val_img_paths = image_paths[split_at:]
val_mask_paths = mask_paths[split_at:]

train_ds = make_tf_dataset(train_img_paths, train_mask_paths, batch_size=BATCH_SIZE, is_training=True)
val_ds   = make_tf_dataset(val_img_paths, val_mask_paths, batch_size=BATCH_SIZE, is_training=False)

print('Train batches:', tf.data.experimental.cardinality(train_ds).numpy())
print('Val batches:', tf.data.experimental.cardinality(val_ds).numpy())


Train batches: 29
Val batches: 8


In [10]:
# 5) Losses & metrics (Dice, BCE+Dice, IoU)
import tensorflow.keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1. - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred, bce_weight=0.5):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return bce_weight * bce + (1.0 - bce_weight) * dice_loss(y_true, y_pred)

def iou_metric(y_true, y_pred, thresh=0.5, smooth=1e-6):
    y_pred_bin = tf.cast(y_pred > thresh, tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred_bin)
    intersect = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersect
    return (intersect + smooth) / (union + smooth)


In [11]:
# 6) Focal loss (binary) option
def binary_focal_loss(gamma=2., alpha=0.25):
    def focal_loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        loss = -alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1) - (1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0)
        return K.mean(loss)
    return focal_loss


In [12]:
# 7) DeepLabV3+ builder (dynamic-shape friendly)
from tensorflow.keras import layers

def ConvBlock(filters=256, kernel_size=3, dilation_rate=1, name=None):
    return keras.Sequential([
        layers.Conv2D(filters, kernel_size, padding='same', dilation_rate=dilation_rate, use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU()
    ], name=name)

def ASPP_block(x, filters=256, name=None):
    conv1 = ConvBlock(filters=filters, kernel_size=1, dilation_rate=1)(x)
    conv6  = ConvBlock(filters=filters, kernel_size=3, dilation_rate=6)(x)
    conv12 = ConvBlock(filters=filters, kernel_size=3, dilation_rate=12)(x)
    conv18 = ConvBlock(filters=filters, kernel_size=3, dilation_rate=18)(x)
    pool = layers.GlobalAveragePooling2D()(x)
    pool = layers.Reshape((1,1, pool.shape[-1]))(pool)
    pool = layers.Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(pool)
    pool = layers.BatchNormalization()(pool)
    pool = layers.ReLU()(pool)
    def resize_to_input(args):
        p, ref = args
        ref_shape = tf.shape(ref)[1:3]
        return tf.image.resize(p, ref_shape, method='bilinear')
    pool = layers.Lambda(resize_to_input)([pool, x])
    concat = layers.Concatenate()([pool, conv1, conv6, conv12, conv18])
    out = ConvBlock(filters=filters, kernel_size=1, dilation_rate=1)(concat)
    return out

def build_deeplabv3plus(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), backbone_weights='imagenet'):
    inputs = layers.Input(shape=input_shape)
    backbone = tf.keras.applications.ResNet50(include_top=False, weights=backbone_weights, input_tensor=inputs)
    low_level_feature = backbone.get_layer('conv2_block3_out').output
    high_level_feature = backbone.get_layer('conv4_block6_out').output
    aspp = ASPP_block(high_level_feature, filters=256)
    def up_to_low(inputs):
        x, ref = inputs
        ref_shape = tf.shape(ref)[1:3]
        return tf.image.resize(x, ref_shape, method='bilinear')
    aspp_up = layers.Lambda(up_to_low)([aspp, low_level_feature])
    low_reduced = layers.Conv2D(48, kernel_size=1, padding='same', use_bias=False)(low_level_feature)
    low_reduced = layers.BatchNormalization()(low_reduced)
    low_reduced = layers.ReLU()(low_reduced)
    concat = layers.Concatenate()([aspp_up, low_reduced])
    x = ConvBlock(filters=256, kernel_size=3)(concat)
    x = ConvBlock(filters=256, kernel_size=3)(x)
    def up_to_input(inputs):
        x, ref = inputs
        ref_shape = tf.shape(ref)[1:3]
        return tf.image.resize(x, ref_shape, method='bilinear')
    x = layers.Lambda(up_to_input)([x, inputs])
    outputs = layers.Conv2D(1, kernel_size=1, activation='sigmoid', name='pred_mask')(x)
    model = keras.models.Model(inputs=inputs, outputs=outputs, name=MODEL_NAME)
    return model

model = build_deeplabv3plus()
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 66s 1us/step 



Model: "DeepLabV3plus_Flood"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 262, 262, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 128, 128, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 128, 128, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 128, 128, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 130, 130, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 64, 64, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 64, 64, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 64, 64, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 64, 64, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 64, 64, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 64, 64, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 64, 64, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 64, 64, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 64, 64, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 17,830,209 (68.02 MB)

 Trainable params: 17,795,425 (67.88 MB)

 Non-trainable params: 34,784 (135.88 KB)

In [13]:
# 8) Compile & callbacks
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=bce_dice_loss, metrics=[dice_coef, tf.keras.metrics.MeanIoU(num_classes=2)])
ckpt = tf.keras.callbacks.ModelCheckpoint(MODEL_NAME + '.h5', save_best_only=True, monitor='val_loss', verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
earlystp  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0)
callbacks = [ckpt, reduce_lr, earlystp, tensorboard]


In [14]:
# 9) Optional: Keras Tuner example (lightweight) - tune learning rate only
try:
    import keras_tuner as kt
    def build_for_tuner(hp):
        m = build_deeplabv3plus(backbone_weights='imagenet')
        lr = hp.Float('lr', 1e-5, 1e-3, sampling='log')
        m.compile(optimizer=tf.keras.optimizers.Adam(lr), loss=bce_dice_loss, metrics=[dice_coef])
        return m
    tuner = kt.RandomSearch(build_for_tuner, objective='val_dice_coef', max_trials=3, overwrite=True, directory='tuner_dir', project_name='deeplab_tune')
    # tuner.search(train_ds, validation_data=val_ds, epochs=8)
    print('Keras Tuner imported; uncomment search line to run tuning (slow).')
except Exception as e:
    print('Keras Tuner not available or failed to import:', e)


Keras Tuner not available or failed to import: No module named 'keras_tuner'


In [ ]:
# 10) Train
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)


Epoch 1/60
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - dice_coef: 0.6008 - loss: 0.4665 - mean_io_u: 0.2927   
Epoch 1: val_loss improved from None to 0.86994, saving model to DeepLabV3plus_Flood.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 259s 9s/step - dice_coef: 0.7059 - loss: 0.3641 - mean_io_u: 0.2924 - val_dice_coef: 0.4779 - val_loss: 0.8699 - val_mean_io_u: 0.3283 - learning_rate: 1.0000e-04
Epoch 2/60
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - dice_coef: 0.7985 - loss: 0.2569 - mean_io_u: 0.3020   
Epoch 2: val_loss improved from 0.86994 to 0.79451, saving model to DeepLabV3plus_Flood.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 246s 8s/step - dice_coef: 0.7959 - loss: 0.2696 - mean_io_u: 0.2924 - val_dice_coef: 0.4766 - val_loss: 0.7945 - val_mean_io_u: 0.3283 - learning_rate: 1.0000e-04
Epoch 3/60
15/29 ━━━━━━━━━━━━━━━━━━━━ 1:43 7s/step - dice_coef: 0.8347 - loss: 0.2152 - mean_io_u: 0.2849

In [ ]:
# 11) Evaluation: compute per-image IoU on validation set
def predict_and_threshold(img_tensor, threshold=0.5):
    pred = model.predict(tf.expand_dims(img_tensor, axis=0))[0]
    mask_bin = (pred[...,0] > threshold).astype(np.uint8)
    return pred[...,0], mask_bin

val_imgs = [load_image(p, size=IMAGE_SIZE, mask=False) for p in val_img_paths]
val_msks = [load_image(p, size=IMAGE_SIZE, mask=True) for p in val_mask_paths]

ious = []
for i in range(len(val_imgs)):
    pred_float, pred_bin = predict_and_threshold(val_imgs[i], threshold=0.5)
    true_bin = (val_msks[i][...,0] > 0.5).astype(np.uint8)
    inter = (true_bin & pred_bin).sum()
    union = (true_bin | pred_bin).sum()
    iou = 1.0 if union == 0 else inter / union
    ious.append(iou)
print('Validation mean IoU:', np.mean(ious))


In [ ]:
# 12) Visualization helpers
def show_sample(img, true_mask, pred_mask, pred_mask_bin=None):
    plt.figure(figsize=(14,4))
    plt.subplot(1,4,1); plt.imshow(img); plt.title('Image'); plt.axis('off')
    plt.subplot(1,4,2); plt.imshow(true_mask.squeeze(), cmap='gray'); plt.title('True Mask'); plt.axis('off')
    plt.subplot(1,4,3); plt.imshow(pred_mask, cmap='gray'); plt.title('Predicted (float)'); plt.axis('off')
    plt.subplot(1,4,4); plt.imshow(img); plt.imshow(pred_mask_bin, cmap='Blues', alpha=0.45); plt.title('Overlay'); plt.axis('off')
    plt.show()

for i in range(min(6, len(val_imgs))):
    pf, pb = predict_and_threshold(val_imgs[i], threshold=0.5)
    show_sample(val_imgs[i], val_msks[i], pf, pb)


In [ ]:
# 13) Grad-CAM style explanation (segmentation-adapted)
def gradcam_for_segmentation(model, img_tensor, layer_name, upsample_to=IMAGE_SIZE):
    img_batch = tf.expand_dims(img_tensor, 0)
    layer = model.get_layer(layer_name).output
    grad_model = tf.keras.models.Model([model.inputs], [layer, model.output])
    with tf.GradientTape() as tape:
        inputs = tf.cast(img_batch, tf.float32)
        tape.watch(inputs)
        feature_maps, preds = grad_model(inputs)
        score = tf.reduce_mean(preds)
    grads = tape.gradient(score, feature_maps)
    weights = tf.reduce_mean(grads, axis=(1,2))
    cam = tf.reduce_sum(tf.multiply(feature_maps, tf.reshape(weights, (1,1,1,-1))), axis=-1)
    cam = cam[0].numpy()
    cam = np.maximum(cam, 0)
    cam = tf.image.resize(cam[..., np.newaxis], upsample_to).numpy().squeeze()
    if cam.max() != cam.min():
        cam = (cam - cam.min()) / (cam.max() - cam.min())
    else:
        cam = np.zeros_like(cam)
    return cam

sample_idx = 0 if len(val_imgs)>0 else None
if sample_idx is not None:
    img = val_imgs[sample_idx]
    try:
        heatmap = gradcam_for_segmentation(model, img, layer_name='conv2d_1')
        plt.figure(figsize=(12,4))
        plt.subplot(1,3,1); plt.imshow(img); plt.title('Image'); plt.axis('off')
        plt.subplot(1,3,2); plt.imshow(heatmap, cmap='jet'); plt.title('Grad-CAM heatmap'); plt.axis('off')
        plt.subplot(1,3,3); plt.imshow(img); plt.imshow(heatmap, cmap='jet', alpha=0.5); plt.title('Overlay'); plt.axis('off')
        plt.show()
    except Exception as e:
        print('GradCAM failed:', e)


In [ ]:
# 14) Save full model
model.save(MODEL_NAME + '_full.h5', include_optimizer=False)
print('Saved model to', MODEL_NAME + '_full.h5')


## Notes
- Adjust `root_path` for your environment.
- Keras Tuner part is optional and can be slow; uncomment to run.
- For production, consider TFRecords and mixed precision for speed.
- You can improve post-processing (morphology, CRF) for sharper boundaries.
